In [0]:
import json# Mount

# Set up the configurations for mounting the GCS bucket
gcs_bucket_name = "bucket1mart"
mount_point = "/mnt/mskltestmnt728"
project_id = "mentorsko-1723044079759"
service_account_key = "/dbfs/FileStore/tables/mentorsko_1723044081443_e08bd1f42ed3.json"

# Read the service account key file
with open(service_account_key, 'r') as key_file:
    service_account_info = json.load(key_file)

# Define the GCS service account credentials
config = {
    "fs.gs.impl": "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem",
    "fs.gs.auth.service.account.enable": "true",
    "fs.gs.auth.service.account.email": service_account_info["client_email"],
    "fs.gs.auth.service.account.private.key.id": service_account_info["private_key_id"],
    "fs.gs.auth.service.account.private.key": service_account_info["private_key"],
    "fs.gs.project.id": project_id
}


# Mount the GCS bucket
dbutils.fs.mount(
    source=f"gs://{gcs_bucket_name}",
    mount_point=mount_point,
    extra_configs=config
)

In [0]:
files = dbutils.fs.ls("/mnt/mskltestmnt785")
display(files)

In [0]:
orders_df = spark.read.csv("dbfs:/mnt/mskltestmnt785/orders.csv", header = True, inferSchema=True)
display(orders_df)

In [0]:
payment_df = spark.read.csv("dbfs:/mnt/mskltestmnt785/payments.csv", header = True, inferSchema=True)
display(payment_df)
payment_methods_df = spark.read.csv("dbfs:/mnt/mskltestmnt785/payment_methods.csv", header = True, inferSchema=True)
display(payment_methods_df)

In [0]:
from pyspark.sql.functions import col

payment_method_preference = {
    payment_df
        .join(payment_methods_df, on = 'PaymentMethodID')
        .groupBy('MethodName')
        .count()
        .orderBy(col('count').desc())
}

payment_method_preference.display()

In [0]:
orders_df.createOrReplaceTempView("orders")

In [0]:
%sql
select
    ProductID,
    sum(Quantity) as TotalQuantitySold
from
  order_items
GROUP BY ProductID
ORDER BY TotalQuantitySold DESC
;
 

In [0]:
orders_df.createOrReplaceGlobalTempView("orders")
payment_df.createOrReplaceTempView("payments")
payment_df.createOrReplaceGlobalTempView("payments")

In [0]:
%sql
select
    o.OrderChannel,
    p.PaymentMethodID,
    count(o.OrderID) as NumberOfOrders
from
  global_temp.orders o
join
  global_temp.payments p on o.OrderID = p.OrderID
GROUP BY o.OrderChannel, p.PaymentMethodID
ORDER BY NumberOfOrders DESC

In [0]:

from pyspark.sql.functions import *

In [0]:
from pyspark.sql.functions import *

# Your Python code here (if any)

# Use %sql for SQL queries
%sql
SELECT c.*, o.*, ot.*, p.*
FROM customers c
JOIN orders o ON c.CustomerID = o.CustomerID
JOIN order_items ot ON o.OrderID = ot.OrderID
JOIN product p ON o.ProductID = p.ProductID